In [7]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords 
nltk.download('stopwords')
import string
string.punctuation
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from copy import deepcopy as dp
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords  
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib
import xgboost as xgb
from sklearn.model_selection import GridSearchCV,train_test_split
import gensim
from gensim.models import Word2Vec
from copy import deepcopy
import multiprocessing
from time import time
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
unable to import 'smart_open.gcs', disabling that module


In [8]:
df1=pd.read_csv('data_train.csv')
df1_test=pd.read_csv('data_test.csv')

In [9]:
len(df1),len(df1_test)

(3834, 956)

In [10]:
df1.head()

,created_utc,link_flair_text,num_comments,score,selftext,subreddit_id,title,url
0,1.553629e+09,3,6,1,"Hi, I'm 16yo from Kandivali, Mumbai. I play ac...",t5_2qh1q,Looking to make friends who are musicians!,https://www.reddit.com/r/india/comments/b5u3or...
1,1.547942e+09,3,67,29,"Hi friends, I'm working on an essay on Indian ...",t5_2qh1q,What bothers you as an Indian?,https://www.reddit.com/r/india/comments/ahrxc7...
2,1.550300e+09,3,4,7,I'm an Indian and I can understand Hindi but c...,t5_2qh1q,What are the best guides to learn to speak Hindi?,https://www.reddit.com/r/india/comments/ar6dvb...
3,1.546852e+09,3,19,21,This post was inspired by another post at the ...,t5_2qh1q,Tips to become a good driver,https://www.reddit.com/r/india/comments/adfxwr...
4,1.547511e+09,3,0,1,[removed],t5_2qh1q,Earning WhatsApp Group Join Link List,https://www.reddit.com/r/india/comments/ag2869...


In [11]:
title=df1['title'].tolist()
body=df1['selftext'].tolist()
url=df1['url'].tolist()
y=df1['link_flair_text'].tolist()
title_test=df1_test['title'].tolist()
body_test=df1_test['selftext'].tolist()
url_test=df1_test['url'].tolist()
y_test=df1_test['link_flair_text'].tolist()

In [12]:

def stopwords_removal(tokens):
    stop_words = list(stopwords.words("english"))
    filtered_sentence = [w for w in tokens if not w in stop_words]              
    final=' '.join(filtered_sentence)
    return final

def remove_punctuation(text):
    obj=str.maketrans('', '',string.punctuation)
    answer=text.translate(obj)
    return answer

def simple_lemma(data):
    lemmatizer = WordNetLemmatizer()
    filtered=[lemmatizer.lemmatize(word) for word in data.split()]
    text = ' '.join(filtered)
    return text

def pre_process(y,flag):
    x=deepcopy(y)
    for i in range(len(x)):
        if str([i])!='nan':
            if flag:
                x[i]=x[i].split('/')
                x[i]=' '.join(x[i])
            x[i]=simple_lemma(remove_punctuation(stopwords_removal(word_tokenize(str(x[i]).lower()))))
        else:
            x[i]=''
    return x

In [13]:
c1=[0 for _ in range(11)]
c2=[0 for _ in range(11)]

In [14]:
for i in df1['link_flair_text'].tolist():
    c1[i]+=1
for i in df1_test['link_flair_text']:
    c2[i]+=1

In [15]:
c1,c2

([199, 500, 500, 500, 500, 500, 41, 344, 244, 406, 100],
 [49, 125, 125, 125, 125, 125, 10, 86, 61, 101, 24])

In [16]:
title_clean=pre_process(title,False)
body_clean=pre_process(body,False)
url_clean=pre_process(url,True)
title_clean_test=pre_process(title_test,False)
body_clean_test=pre_process(body_test,False)
url_clean_test=pre_process(url_test,True)

In [17]:
title[0],title_clean[0]

('Looking to make friends who are musicians!', 'looking make friend musician')

In [29]:
print(body[3])
print("\n")
print(body_clean[3])

This post was inspired by another post at the top today - https://www.reddit.com/r/india/comments/addzf5/why_does_everyone_have_their_highbeams_on/

And this comment especially,

&gt; Because no one taught them how high beams work or how pass lights work or how turn indicators work or...

This is me. I wasn't taught well and I have a inherited a lot of bad driving habits that I now want to correct.

Please help me become a responsible driver.


post inspired another post top today http wwwredditcomrindiacommentsaddzf5whydoeseveryonehavetheirhighbeamson comment especially gt one taught high beam work pas light work turn indicator work nt taught well inherited lot bad driving habit want correct please help become responsible driver


In [30]:
def train(classifier,X_train,y_train,X_test,y_test):
    classifier.fit(X_train,y_train)
    y_train_pred=classifier.predict(X_train)
    y_test_pred=classifier.predict(X_test)
    return accuracy_score(y_train,y_train_pred),accuracy_score(y_test,y_test_pred)

In [31]:

def train_test(X_train,X_test):
    tfidf= TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
    X1_train=tfidf.fit_transform(X_train)
    X1_test=tfidf.transform(X_test)
    
    clf1=GridSearchCV(SVC(gamma='scale'),{'decision_function_shape':['ovo','ovr'],},cv=5,iid=False)
    print(train(clf1,X1_train,y,X1_test,y_test))
    
    clf2=GridSearchCV(DecisionTreeClassifier(random_state=0),{'criterion':['gini','entropy']},cv=5,iid=False)
    print(train(clf2,X1_train,y,X1_test,y_test))

    clf3=GridSearchCV(LogisticRegression(random_state=0),{'solver':['newton-cg', 'lbfgs', 'saga'],'multi_class':['multinomial'],'max_iter':[500,1000,1500]},cv=5,iid=False)
    print(train(clf3,X1_train,y,X1_test,y_test))

    clf4=GridSearchCV(RandomForestClassifier(random_state=0),{'n_estimators':[10,50,100,150],'criterion':['gini','entropy']},cv=5,iid=False)
    print(train(clf4,X1_train,y,X1_test,y_test))

    clf5=GridSearchCV(xgb.XGBClassifier(objective="binary:logistic", random_state=42),{'max_depth':[2,3],'learning_rate':[0.1,0.15,0.2]},cv=5,iid=False)
    print(train(clf5,X1_train,y,X1_test,y_test))
    
    

In [35]:
train_test(title_clean,title_clean_test)

(0.8873239436619719, 0.4884937238493724)
(0.9577464788732394, 0.3880753138075314)
(0.724830464267084, 0.4884937238493724)
(0.9577464788732394, 0.47384937238493724)
(0.6674491392801252, 0.44037656903765693)


In [33]:
train_test(url_clean,url_clean_test)

(0.4590505998956703, 0.3985355648535565)
(0.4809598330725091, 0.3797071129707113)
(0.4428794992175274, 0.39644351464435146)
(0.4809598330725091, 0.3891213389121339)
(0.45539906103286387, 0.39330543933054396)


In [34]:
train_test(body_clean,body_clean_test)

(0.3075117370892019, 0.23326359832635984)
(0.32785602503912364, 0.2395397489539749)
(0.2540427751695357, 0.24476987447698745)
(0.32785602503912364, 0.2510460251046025)
(0.28351591027647366, 0.23744769874476987)


In [36]:
ti_bo=[]
ti_bo_test=[]
for i in range(len(title_clean)):
    ti_bo.append(title_clean[i]+" "+body_clean[i])
for i in range(len(title_clean_test)):
    ti_bo_test.append(title_clean_test[i]+' '+body_clean_test[i])
train_test(ti_bo,ti_bo_test)
print(len(title_clean),len(title_clean_test),len(body_clean_test))

(0.9290558163797601, 0.5240585774058577)
(0.9754825247782994, 0.37343096234309625)
(0.7556077203964527, 0.5313807531380753)
(0.9754825247782994, 0.49372384937238495)
(0.7112676056338029, 0.49581589958158995)
3834 956 956


In [37]:
ti_bo_ur=[]
for i in range(len(title_clean)):
    ti_bo_ur.append(title_clean[i]+' '+body_clean[i]+' '+url_clean[i])
ti_bo_ur_test=[]
for i in range(len(title_clean_test)):
    ti_bo_ur_test.append(title_clean_test[i]+' '+body_clean_test[i]+' '+url_clean_test[i])
train_test(ti_bo_ur,ti_bo_ur_test)

(0.9264475743348983, 0.5575313807531381)
(0.985133020344288, 0.42887029288702927)
(0.7496087636932708, 0.5700836820083682)
(0.985133020344288, 0.5449790794979079)
(0.7378716744913928, 0.5753138075313807)


In [38]:
bo_ur=[]
for i in range(len(body_clean)):
    bo_ur.append(body_clean[i]+' '+url_clean[i])
bo_ur_test=[]
for i in range(len(body_clean_test)):
    bo_ur_test.append(body_clean_test[i]+' '+url_clean_test[i])
train_test(bo_ur,bo_ur_test)

(0.5878977569118414, 0.4299163179916318)
(0.634585289514867, 0.399581589958159)
(0.5179968701095462, 0.44142259414225943)
(0.634585289514867, 0.4361924686192469)
(0.548774126238915, 0.4320083682008368)


In [39]:
ti_ur=[]
for i in range(len(title_clean)):
    ti_ur.append(title_clean[i]+' '+url_clean[i])
ti_ur_test=[]
for i in range(len(title_clean_test)):
    ti_ur_test.append(title_clean_test[i]+' '+url_clean_test[i])
train_test(ti_ur,ti_ur_test)

(0.8951486697965572, 0.5428870292887029)
(0.9778299426186751, 0.4581589958158996)
(0.7516953573291602, 0.5658995815899581)
(0.9778299426186751, 0.5219665271966527)
(0.6987480438184663, 0.5313807531380753)


In [40]:
clf=GridSearchCV(SVC(gamma='scale'),{'decision_function_shape':['ovo','ovr'],},cv=5,iid=False)

In [41]:
tfidf= TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidf.fit_transform(ti_bo_ur)  
    

In [42]:
clf.fit(X,y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'decision_function_shape': ['ovo', 'ovr']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [43]:
y_pred=clf.predict(X)
print(accuracy_score(y,y_pred))

0.9264475743348983


In [45]:
with open('Model','wb') as f:
    pickle.dump(clf,f)

In [46]:
with open('tfidf','wb') as f:
    pickle.dump(tfidf,f)
    